In [1]:
import gensim.downloader as api
from sklearn.datasets import fetch_20newsgroups
import torch

from experiments.experiment import snn_experiment, lsa_experiment
from experiments.preprocess import Word2VecPrep

In [2]:
device = "cuda" if torch.cuda.is_available else "cpu"

In [3]:
word2vec = api.load('word2vec-google-news-300')

In [4]:
cats = ['comp.graphics', 'sci.med']  #, 'talk.politics.guns']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

text_prep = Word2VecPrep(word2vec)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aleks\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
newsgroups_train.data[:10]

['From: zyeh@caspian.usc.edu (zhenghao yeh)\nSubject: Re: Need polygon splitting algo...\nOrganization: University of Southern California, Los Angeles, CA\nLines: 25\nDistribution: world\nNNTP-Posting-Host: caspian.usc.edu\nKeywords: polygons, splitting, clipping\n\n\nIn article <1qvq4b$r4t@wampyr.cc.uow.edu.au>, g9134255@wampyr.cc.uow.edu.au (Coronado Emmanuel Abad) writes:\n|> \n|> The idea is to clip one polygon using another polygon (not\n|> necessarily rectangular) as a window.  My problem then is in\n|> finding out all the new vertices of the resulting "subpolygons"\n|> from the first one.  Is this simply a matter of extending the\n|> usual algorithm whereby each of the edges of one polygon is checked\n|> against another polygon???  Is there a simpler way??\n|> \n|> Comments welcome.\n|> \n|> Noel.\n\n\tIt depends on what kind of the polygons. \n\tConvex - simple, concave - trouble, concave with loop(s)\n\tinside - big trouble.\n\n\tOf cause, you can use the box test to avoid che

In [6]:
train_x, train_y = text_prep.preprocess_dataset(newsgroups_train, avg=True)
train_x.to(device)
train_y.to(device)
print(device)

c:\Users\aleks\Documents\Moje dokumenty\Studia\Praca Magisterska\NeuCube-Py-TextRep\experiments\preprocess.py:85: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  train_x = tensor(newsgroups_vectors)


cuda


In [7]:
lsa_experiment(train_x, train_y, clf_type="regression", splits=5)

5it [00:00, 20.75it/s]


---- CLASSIFIER: regression ----
acc: 0.9634974533106961
[[574  10]
 [ 33 561]]


In [8]:
#lsa_experiment(train_x, train_y, clf_type="random_forest", splits=8)

In [9]:
#lsa_experiment(train_x, train_y, clf_type="xgboost", splits=8)

In [10]:
#encoder = Probability(iterations=100)
#spike_train_x = encoder.encode_dataset(newsgroups_vectors_tensor)

In [15]:
spike_train_x, spike_train_y = text_prep.preprocess_dataset(newsgroups_train, avg=False, max_size=100)

In [16]:
#mask = (spike_train_x > -0.1) & (spike_train_x < 0.1)
mask = spike_train_x >= 0.25
data_x = torch.where(mask, torch.tensor(1), torch.tensor(0))

In [17]:
data_x[0][9]

tensor([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [18]:
data_x.to(device)
spike_train_y.to(device)
print(device)

cuda


In [20]:
snn_experiment(data_x, spike_train_y, clf_type="regression", splits=5, shape=(4, 4, 4), res_train=True)

5it [10:12, 122.47s/it]

---- CLASSIFIER: regression ----
acc: 0.8514431239388794
[[505  79]
 [ 96 498]]


In [21]:
snn_experiment(data_x, spike_train_y, clf_type="regression", splits=5, shape=(5, 5, 5), res_train=True)

5it [10:06, 121.25s/it]

---- CLASSIFIER: regression ----
acc: 0.8752122241086587
[[507  77]
 [ 70 524]]


In [22]:
snn_experiment(data_x, spike_train_y, clf_type="regression", splits=5, shape=(6, 6, 6), res_train=True)

5it [10:14, 122.94s/it]

---- CLASSIFIER: regression ----
acc: 0.8938879456706282
[[529  55]
 [ 70 524]]


In [23]:
snn_experiment(data_x, spike_train_y, clf_type="regression", splits=5, shape=(5, 6, 7), res_train=True)

5it [10:15, 123.16s/it]

---- CLASSIFIER: regression ----
acc: 0.9015280135823429
[[530  54]
 [ 62 532]]


In [19]:
snn_experiment(data_x, spike_train_y, clf_type="regression", splits=5, shape=(8, 8, 8), res_train=True)

5it [10:34, 126.82s/it]

---- CLASSIFIER: regression ----
acc: 0.9142614601018676
[[537  47]
 [ 54 540]]
